In [2]:
from dask_jobqueue import SLURMCluster 
from dask.distributed import Client 
  
cluster = SLURMCluster(cores=28,name='make_profiles',walltime='00:30:00',job_extra=['--constraint=HSW24','--exclusive','--nodes=1'],memory='120GB',interface='ib0') 

  

/scratch/cnt0024/hmg2840/albert7a/anaconda3/lib/python3.7/site-packages/distributed/dashboard/core.py:74: UserWarning: 
Failed to start diagnostics server on port 8787. [Errno 13] Permission denied
  warnings.warn("\n" + msg)


In [3]:
cluster.scale(102) 


In [4]:
from dask.distributed import Client
client = Client(cluster)


In [5]:
client


Client Scheduler: tcp://172.30.100.2:44391 Dashboard: http://172.30.100.2:43265/status,Cluster Workers: 0 Cores: 0 Memory: 0 B


In [6]:
import xarray as xr 
import dask 
import numpy as np 
import os 
import time 
import glob
import datetime
import pandas as pd
import sys

today=datetime.date.today()

import sys
sys.path.insert(0,'/home/albert7a/git/xscale')
import xscale

In [7]:
data_dir = '/store/CT1/hmg2840/lbrodeau/eNATL60/eNATL60-BLBT02-S/'
gridfile='/store/CT1/hmg2840/lbrodeau/eNATL60/eNATL60-I/mesh_mask_eNATL60_3.6_lev1.nc4'
dsgrid=xr.open_dataset(gridfile,chunks={'x':1000,'y':1000})


In [8]:
def read_csv(box):
    boxes=pd.read_csv('/home/albert7a/git/formation_ANNA/make_boxes/boxes_'+str(box)+'_1x1_eNATL60.csv',sep = '\t',index_col=0)
    imin=boxes['imin']
    imax=boxes['imax']
    jmin=boxes['jmin']
    jmax=boxes['jmax']
    box_name=boxes.index
    return imin,imax,jmin,jmax,box_name

def compute_buoy(t,s):
    rau0  = 1000
    grav  = 9.81
    buoy= -1*(grav/rau0)*sigma0(t,s)
    return buoy

def sigma0(t,s):
    zrau0=1000
    zsr=np.sqrt(np.abs(s))
    zs=s
    zt=t
    zr1 = ( ( ( ( 6.536332e-9*zt-1.120083e-6 )*zt+1.001685e-4)*zt - 9.095290e-3 )*zt+6.793952e-2 )*zt+999.842594
    zr2= ( ( ( 5.3875e-9*zt-8.2467e-7 )*zt+7.6438e-5 ) *zt - 4.0899e-3 ) *zt+0.824493
    zr3= ( -1.6546e-6*zt+1.0227e-4 ) *zt-5.72466e-3
    zr4= 4.8314e-4
    sigma0=( zr4*zs + zr3*zsr + zr2 ) *zs + zr1 - zrau0
    return sigma0

def filt(w):
    win_box2D = w.window
    win_box2D.set(window='hanning', cutoff=20, dim=['x', 'y'], n=[30, 30])
    bw = win_box2D.boundary_weights(drop_dims=[])
    w_LS = win_box2D.convolve(weights=bw)
    w_SS=w-w_LS
    return w_SS


In [9]:
date='20090701'
print('read the data')
tfilename = sorted(glob.glob(data_dir+'*/eNATL60-BLBT02_1h_*_gridT_'+date+'-'+date+'.nc'))
tfile=tfilename[0]
dst=xr.open_dataset(tfile,chunks={'x':1000,'y':1000,'time_counter':1,'deptht':1})
tdata=dst['votemper']
sfilename = sorted(glob.glob(data_dir+'*/eNATL60-BLBT02_1h_*_gridS_'+date+'-'+date+'.nc'))
sfile=sfilename[0]
dss=xr.open_dataset(sfile,chunks={'x':1000,'y':1000,'time_counter':1,'deptht':1})
sdata=dss['vosaline']
wfilename = sorted(glob.glob(data_dir+'*/eNATL60-BLBT02_1h_*_gridW_'+date+'-'+date+'.nc'))
wfile=wfilename[0]
dsw=xr.open_dataset(wfile,chunks={'x':1000,'y':1000,'time_counter':1,'depthw':1})
wdata=dsw['vovecrtz']
wdata_t=wdata.rename({'depthw':'deptht'})
print('compute buoyancy')
buoy=compute_buoy(tdata,sdata)
print('filter w')
wprime=filt(wdata_t)
print('filter buoyancy')
bprime=filt(buoy)


read the data
compute buoyancy
filter w
filter buoyancy


distributed.utils_perf - WARNING - full garbage collections took 30% CPU time recently (threshold: 10%)


In [10]:
deptht=dst.deptht
wdata_t=wdata.reindex({'depthw':deptht},method='nearest')
wdata_t=wdata_t.rename({'depthw':'deptht'})

/scratch/cnt0024/hmg2840/albert7a/anaconda3/lib/python3.7/site-packages/xarray/core/dataarray.py:979: FutureWarning: Indexer has dimensions ('deptht',) that are different from that to be indexed along depthw. This will behave differently in the future.
  indexers=indexers, method=method, tolerance=tolerance, copy=copy)


In [11]:
wprime=filt(wdata_t)


distributed.utils_perf - WARNING - full garbage collections took 32% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took 33% CPU time recently (threshold: 10%)


In [12]:
wprimebprime=wprime*bprime

In [ ]:
cluster.close()
client.close()

In [13]:
## box indices 
boxes_LS=pd.read_csv('/home/albert7a/git/formation_ANNA/make_boxes/boxes_LS_1x1_eNATL60.csv',sep = '\t',index_col=0)
imin_LS=boxes_LS['imin']
imax_LS=boxes_LS['imax']
jmin_LS=boxes_LS['jmin']
jmax_LS=boxes_LS['jmax']
box_name=boxes_LS.index



In [14]:
imin=imin_LS[0]
imax=imax_LS[0]
jmin=jmin_LS[0]
jmax=jmax_LS[0]


In [15]:
print(imin,imax,jmin,jmax)

2698 2759 3601 3664


In [ ]:
wprimebprime_LS=wprimebprime[:,:,jmin:jmax,imin:imax]
dataset=wprimebprime_LS.to_dataset(name='wprimebprime')
attrs=tdata.attrs
dataset['wprimebprime'].attrs=attrs
dataset['wprimebprime'].attrs['standard_name']='product of filtered vertical velocity and buoyancy'
dataset['wprimebprime'].attrs['long_name']='wprimebprime'
dataset['wprimebprime'].attrs['units']=''        
dataset.attrs['global_attribute']= 'product of filtered vertical velocity and buoyancy in box LS'
dataset.to_netcdf(path='/scratch/cnt0024/hmg2840/albert7a/eNATL60/eNATL60-BLBT02-S/ANNA/LS/eNATL60LS-BLBT02_y'+date[0:4]+'m'+date[4:6]+'d'+date[6:9]+'_wprimebprime.nc',mode='w')


distributed.utils_perf - WARNING - full garbage collections took 34% CPU time recently (threshold: 10%)
